In [1]:
# The main idea is to check the market depth of currencies through KuCoin exchange. I want to check, how price could change with putting 50K USD in instant market order and try to export coins with lowest market depth.

In [6]:
#Neccessary library for API commands

In [5]:
pip install kucoin-python  

Note: you may need to restart the kernel to use updated packages.


In [7]:
from kucoin.market import market
import os # Co to za biblioteka
import pandas as pd
import numpy as np
api_key = '63a8ab5fce40be0001afdb14'
api_secret = '4a1cea16-6771-4456-94ae-6479334406fb'
api_passphrase = '-AMCtothemoon!!!'

In [8]:
market = market.MarketData(key = api_key, secret = api_secret, passphrase = api_passphrase)


In [10]:
# first im gonna try to an operation with single coin

In [11]:
order_book = market.get_aggregated_orderv3('BTC-USDT')  # API documentation


In [12]:
order_book.keys()

dict_keys(['time', 'sequence', 'bids', 'asks'])

In [13]:
type(order_book)

dict

In [14]:
# 'asks' are neccessary for further operation

In [15]:
ask_btc = order_book['asks']

In [16]:
ask_btc[:5]

[['16815.6', '3.90663011'],
 ['16816.7', '0.49999999'],
 ['16816.8', '0.27750152'],
 ['16816.9', '0.90861867'],
 ['16817', '0.002']]

In [17]:
type(ask_btc)

list

In [18]:
type(ask_btc[0][1])

str

In [19]:
btc_df = pd.DataFrame(ask_btc, columns = ['price','amount'])  # df for adding column 'price'*'amount'

In [20]:
btc_df.head()

,price,amount
0,16815.6,3.90663011
1,16816.7,0.49999999
2,16816.8,0.27750152
3,16816.9,0.90861867
4,16817,0.002


In [21]:
btc_df['price'] = btc_df['price'].astype(float)  # we need floats for multiple columns 'price'*'amount'
btc_df['amount'] = btc_df['amount'].astype(float)   

In [22]:
type(btc_df['price'][0])

numpy.float64

In [24]:
type(btc_df['amount'][0])

numpy.float64

In [26]:
btc_df['sum'] = btc_df['price']*btc_df['amount'] 

In [33]:
btc_df.drop('ask', axis = 'columns', inplace = True)

In [34]:
btc_df.head()

,price,amount,sum
0,16815.6,3.906630,65692.329278
1,16816.7,0.500000,8408.349832
2,16816.8,0.277502,4666.687562
3,16816.9,0.908619,15280.149312
4,16817.0,0.002000,33.634000


In [36]:
type(btc_df['sum'][0])

numpy.float64

In [38]:
btc_list = btc_df.values.tolist() #change to list for faster calculations

In [39]:
btc_list[:5]

[[16815.6, 3.90663011, 65692.329277716],
 [16816.7, 0.49999999, 8408.349831833],
 [16816.8, 0.27750152, 4666.6875615359995],
 [16816.9, 0.90861867, 15280.149311523],
 [16817.0, 0.002, 33.634]]

In [40]:
len(btc_list)

26952

In [41]:
# I want to check, on which order through orderbook we are out of money - it means, that we gonna buy order after order till we spent whole 50K - with this way we gonna find a future price

In [42]:
actual_price_btc = btc_list[0][0]
cash = 2000000
result = []
for i in range(len(btc_list)):
    b = cash - btc_list[i][2]
    cash = b
    if cash <=0:
        result.append([actual_price_btc, btc_list[i][0], (btc_list[i][0]/actual_price_btc)/100])
        break

In [43]:
print(result)

[[16815.6, 16955.8, 0.010083374961345417]]


In [44]:
####### Now we gonna do the same operation for rest coins 

In [45]:
from kucoin.client import Market

In [46]:
client = Market(url='https://api.kucoin.com')  # API documentation

In [47]:
tickers = client.get_all_tickers()  #we need all tickers for make a list with USDT tickers

In [48]:
tickers.keys()

dict_keys(['time', 'ticker'])

In [49]:
tickers = pd.DataFrame(tickers['ticker'])

In [50]:
tickers.head()

,symbol,symbolName,buy,sell,changeRate,changePrice,high,low,vol,volValue,last,averagePrice,takerFeeRate,makerFeeRate,takerCoefficient,makerCoefficient
0,NKN-USDT,NKN-USDT,0.083653,0.083985,0.0275,0.002255,0.085,0.081398,98953.6657,8234.72134534,0.083978,0.08179213,0.001,0.001,1,1
1,LOOM-BTC,LOOM-BTC,0.000002541,0.000002565,-0.0062,-0.000000016,0.00000257,0.000002537,6733.27,0.0171576657159,0.000002544,0.00000258,0.001,0.001,1,1
2,GEM-USDT,GEM-USDT,0.01746,0.01747,-0.0432,-0.00079,0.01886,0.01687,2265729.3866,40332.97507486,0.01746,0.01923083,0.001,0.001,2,2
3,CUSD-USDT,CUSD-USDT,0.9959,0.996,0.0005,0.0005,0.9964,0.995,18178.1067,18098.22621499,0.996,0.99647303,0.001,0.001,2,2
4,LTC3L-USDT,LTC3L-USDT,0.001488,0.001491,-0.0273,-0.000042,0.00161,0.001479,24441098.3934,37692.696220094,0.001491,0.00149236,0.001,0.001,1,1


In [51]:
tickers.set_index('symbol', inplace=True)

In [52]:
tickers.head()

,symbolName,buy,sell,changeRate,changePrice,high,low,vol,volValue,last,averagePrice,takerFeeRate,makerFeeRate,takerCoefficient,makerCoefficient
symbol,,,,,,,,,,,,,,,
NKN-USDT,NKN-USDT,0.083653,0.083985,0.0275,0.002255,0.085,0.081398,98953.6657,8234.72134534,0.083978,0.08179213,0.001,0.001,1,1
LOOM-BTC,LOOM-BTC,0.000002541,0.000002565,-0.0062,-0.000000016,0.00000257,0.000002537,6733.27,0.0171576657159,0.000002544,0.00000258,0.001,0.001,1,1
GEM-USDT,GEM-USDT,0.01746,0.01747,-0.0432,-0.00079,0.01886,0.01687,2265729.3866,40332.97507486,0.01746,0.01923083,0.001,0.001,2,2
CUSD-USDT,CUSD-USDT,0.9959,0.996,0.0005,0.0005,0.9964,0.995,18178.1067,18098.22621499,0.996,0.99647303,0.001,0.001,2,2
LTC3L-USDT,LTC3L-USDT,0.001488,0.001491,-0.0273,-0.000042,0.00161,0.001479,24441098.3934,37692.696220094,0.001491,0.00149236,0.001,0.001,1,1


In [53]:
list_tickers = list(tickers.index.values)

In [54]:
list_tickers[:5]

['NKN-USDT', 'LOOM-BTC', 'GEM-USDT', 'CUSD-USDT', 'LTC3L-USDT']

In [55]:
usdt_tickers = []
for i in list_tickers:
    if 'USDT' in i:
        usdt_tickers.append(i)    # only USDT tickers for further operations

In [56]:
usdt_tickers[:10]

['NKN-USDT',
 'GEM-USDT',
 'CUSD-USDT',
 'LTC3L-USDT',
 'OAS-USDT',
 'KNC-USDT',
 'LYM-USDT',
 'HAI-USDT',
 'MITX-USDT',
 'PDEX-USDT']

In [57]:
len(usdt_tickers)

765

In [65]:
orderbook_usdt = []
for i in usdt_tickers:
    orderbook_usdt.append(market.get_aggregated_orderv3(i))

Exception: 429-{"code":"429000","msg":"Too Many Requests"}

In [66]:
len(orderbook_usdt)

29

In [87]:
orderbook_usdt = orderbook_usdt[:-1]

In [68]:
len(orderbook_usdt)

28

In [88]:
orderbook_usdt[:1]

[{'time': 1672154292521,
  'sequence': '19847310',
  'bids': [['0.083761', '2234.4487'],
   ['0.083737', '2325'],
   ['0.083702', '24576.5122'],
   ['0.083611', '3052'],
   ['0.083601', '7.8256'],
   ['0.083577', '1.7906'],
   ['0.083536', '67.727'],
   ['0.083516', '416.4568'],
   ['0.083485', '13260'],
   ['0.083428', '2.2521'],
   ['0.083418', '1.1988'],
   ['0.083412', '2129.4959'],
   ['0.083348', '7.8494'],
   ['0.083321', '1.7961'],
   ['0.083282', '67.9336'],
   ['0.083263', '1.8578'],
   ['0.083262', '415.8694'],
   ['0.083225', '44.2637'],
   ['0.083132', '10.5059'],
   ['0.083095', '7.8733'],
   ['0.083064', '1.8016'],
   ['0.083029', '1.1406'],
   ['0.083026', '1.2045'],
   ['0.08301', '1.8635'],
   ['0.083009', '417.1369'],
   ['0.082878', '2.2521'],
   ['0.082842', '7.8973'],
   ['0.082808', '1.8072'],
   ['0.082777', '68.348'],
   ['0.082758', '1.8692'],
   ['0.082757', '418.4071'],
   ['0.082729', '44.2637'],
   ['0.082725', '23.8873'],
   ['0.082636', '1.2102'],
   ['0

In [96]:
orderbook_usdt_asks = []
for i in orderbook_usdt:
       orderbook_usdt_asks.append(i['asks'])

In [70]:
# convert every single orderbook to seperated df

In [100]:
orderbook_usdt_df = []
for i in orderbook_usdt_asks:
    a = pd.DataFrame(i, columns = ['price','amount'])
    orderbook_usdt_df.append(a)

In [71]:
# whole strings to floats for 'sum' column

In [101]:
for i in orderbook_usdt_df:
    i['price'] = i['price'].astype(float)
    i['amount'] = i['amount'].astype(float)

In [105]:
type(orderbook_usdt_df[0]['price'][0])

numpy.float64

In [106]:
#adding column 'sum' to every coins df

In [107]:
for i in orderbook_usdt_df:
    i['sum'] = i['price']*i['amount']

In [110]:
# moving back all to list  

In [111]:
orderbook_usdt_asks_list = []
for i in orderbook_usdt_df:
    a = i.values.tolist()
    orderbook_usdt_asks_list.append(a)

In [112]:
# what happen to every coin after putting 50k $ immadiatelely

In [113]:
final_list = []
for i in orderbook_usdt_asks_list:
    cash = 50000
    actual_price = i[0][0]
    for j in i:
        b = cash - j[2]
        cash = b
        if cash <=0:
            final_list.append([actual_price,j[0],(j[0]/actual_price)*100])
            break

In [114]:
final_list

[[0.083985, 0.210901, 251.11746145144966],
 [0.01747, 0.042, 240.41213508872357],
 [0.996, 20.5, 2058.2329317269073],
 [0.001513, 2.399, 158559.15399867814],
 [0.05359, 0.084, 156.74566150401193],
 [0.53, 2.135, 402.83018867924517],
 [0.001181, 0.0099, 838.2726502963591],
 [0.01573, 0.08, 508.5823267641449],
 [0.003005, 0.0065, 216.30615640599],
 [1.5843, 4.6311, 292.31206210944896],
 [0.02032, 0.035, 172.24409448818898],
 [0.000297, 0.00114, 383.83838383838383],
 [0.0211, 0.07627, 361.4691943127962],
 [0.00018435, 0.001014, 550.0406834825061],
 [0.2634, 3.0276, 1149.4305239179953],
 [0.073461, 0.89, 1211.5272049114496],
 [0.1286, 3.9, 3032.659409020218],
 [0.224, 0.988, 441.07142857142856],
 [0.138686, 0.338, 243.7160203625456],
 [0.04569, 0.062, 135.697089078573],
 [1.082, 2.882, 266.358595194085],
 [4.86, 7.0, 144.03292181069958],
 [6e-05, 0.00055, 916.6666666666667],
 [0.003712, 0.039655, 1068.292025862069],
 [0.00953, 0.05, 524.6589716684156],
 [2.334e-05, 9.17e-05, 392.8877463581

In [122]:
usdt_tickers1 = usdt_tickers[:27]

In [123]:
len(usdt_tickers1)

27

In [119]:
final_df = pd.DataFrame(final_list, columns = ['first_price','final_price','% change'])

In [120]:
final_df['% change'] = final_df['% change']/100

In [124]:
final_df['Coin'] = [i for i in usdt_tickers1]

In [130]:
final_df.sort_values(by= '% change', ascending = False)

,first_price,final_price,% change,Coin
3,0.001513,2.399000,1585.591540,LTC3L-USDT
26,0.018319,2.000000,109.176265,ATOM3S-USDT
16,0.128600,3.900000,30.326594,POWR-USDT
2,0.996000,20.500000,20.582329,CUSD-USDT
15,0.073461,0.890000,12.115272,YLD-USDT
14,0.263400,3.027600,11.494305,AGLD-USDT
23,0.003712,0.039655,10.682920,BULL-USDT
22,0.000060,0.000550,9.166667,IHC-USDT
6,0.001181,0.009900,8.382727,LYM-USDT
13,0.000184,0.001014,5.500407,ATOM3L-USDT
